# Análisis RFM

El análisis RFM es un modelo que permite analizar el comportamiento de los clientes, creando segmentos internamente homogéneos y mutuamente heterogéneos a partir de su comportamiento observado en los tickets de transacciones.

El análisis RFM implica el uso de tres variables: 
1. Recencia (actualidad): Tiempo que ha transcurrido desde la última compra. Generalmente se considera que esta es la variable más importante para la segmentación.
2. Frecuencia: Número de compras en un periodo específico de tiempo
3. Monetario: Cantidad de dinero gastada en un periodo específico de tiempo. Una alternativa también es considerar el monto promedio por compra para evitar colinealidad con la frecuencia.


Cada variable generalmente es evaluada en cinco niveles con base en quintiles (20% de los datos). En esta escala más es mejor, por ejemplo, entre menor es el tiempo desde la última compra, el valor en la escala es más alto.


Las tres variables son combinadas para crear una clasificación de tres dígitos. Por ejemplo, un cliente cuya última compra está en el 20% más reciente, que está en el 20% que más veces compra, y que está en el 20% que más ha comprado, tendría una clasificación 5-5-5.

Existen otras variantes del modelo RFM utilizando valores promedio, valores z, ponderando las variables, clasificando en dos etapas y otras alternativas más. Sin embargo, de momento no se abordarán estas variantes.

Para aplicar el análisis RFM las empresas deben tener un identificador del cliente y almacenar información de fecha y monto de cada transacción. Como ventajas del modelo RFM están su costo-beneficio, facilidad en su procedimiento y comprensión, estimación de la rentabilidad de los clientes y segmentos, y así su utilidad para el establecimiento de estrategias. Como desventajas, es un modelo principalmente descriptivo, tiene falta de precisión y presenta variabilidad entre empresas.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_excel('data/b16_rfm.xlsx')

In [ ]:
df.info()

In [ ]:
# Calcular recencia (recency)
ultima_fecha = max(df['Fecha'])
recencia_df = df.groupby('Cliente')['Fecha'].max().reset_index()
recencia_df['recencia'] = (ultima_fecha - recencia_df['Fecha']).dt.days
recencia_df.tail(3)

In [ ]:
# Calcular frecuencia (frequency)
frecuencia_df = df.groupby('Cliente').size().reset_index(name='frecuencia')
frecuencia_df.tail(3)

In [ ]:
# Calcular monto (monetary)
monto_df= df.groupby('Cliente')['Importe'].sum().reset_index(name='monto')
monto_df.tail(3)

In [ ]:
# Obtener quintiles
recencia_df['R'] = pd.qcut(recencia_df['recencia'], 5, labels=["5", "4", "3", "2", "1"])
frecuencia_df['F'] = pd.qcut(frecuencia_df['frecuencia'], 5, labels=["1", "2", "3", "4", "5"])
monto_df['M'] = pd.qcut(monto_df['monto'], 5, labels=["1", "2", "3", "4", "5"])

In [ ]:
# Calcular puntaje RFM
rfm = pd.merge(recencia_df[['Cliente', 'R']], frecuencia_df[['Cliente', 'F']], on='Cliente')
rfm = pd.merge(rfm, monto_df[['Cliente', 'M']], on='Cliente')

In [ ]:
rfm["RF_SCORE"] = rfm['R'].astype(str) + rfm['F'].astype(str)
rfm.tail()

In [ ]:
rfm.info()

In [ ]:
seg_map = {
    r'[1-2][1-2]': 'Hibernando',
    r'[1-2][3-4]': 'En riesgo',
    r'[1-2]5': 'No se debe perder',
    r'3[1-2]': 'Cerca de suspender',
    r'33': 'Necesita atención',
    r'[3-4][4-5]': 'Clientes leales',
    r'41': 'Prometedores',
    r'51': 'Nuevos clientes',
    r'[4-5][2-3]': 'Potencialmente leales',
    r'5[4-5]': 'Campeones'}
rfm["segment"] = rfm["RF_SCORE"].replace(seg_map, regex=True)
rfm.tail()